In [1]:
from google.colab import files

# Upload the Excel file
uploaded = files.upload()



Saving Copy of Analytics Position Case Study.xlsx to Copy of Analytics Position Case Study.xlsx


In [2]:
import pandas as pd

# Load the uploaded Excel file
file_path = "Copy of Analytics Position Case Study.xlsx"
excel_file = pd.ExcelFile(file_path)

# Clean Gameplay Data
gameplay_df = excel_file.parse("User Gameplay data").iloc[3:].copy()
gameplay_df.columns = ['User ID', 'Games Played', 'Datetime']
gameplay_df['User ID'] = gameplay_df['User ID'].astype(int)
gameplay_df['Games Played'] = gameplay_df['Games Played'].astype(int)
games_played = gameplay_df.groupby('User ID')['Games Played'].sum()

# Clean Deposit Data
deposit_df = excel_file.parse("Deposit Data").iloc[3:].copy()
deposit_df.columns = ['User ID', 'Datetime', 'Amount']
deposit_df['User ID'] = deposit_df['User ID'].astype(int)
deposit_df['Amount'] = deposit_df['Amount'].astype(float)
deposits = deposit_df.groupby('User ID').agg(
    total_deposit_amount=('Amount', 'sum'),
    num_deposits=('Amount', 'count')
)

# Clean Withdrawal Data
withdrawal_df = excel_file.parse("Withdrawal Data").iloc[3:].copy()
withdrawal_df.columns = ['User ID', 'Datetime', 'Amount']
withdrawal_df['User ID'] = withdrawal_df['User ID'].astype(int)
withdrawal_df['Amount'] = withdrawal_df['Amount'].astype(float)
withdrawals = withdrawal_df.groupby('User ID').agg(
    total_withdrawal_amount=('Amount', 'sum'),
    num_withdrawals=('Amount', 'count')
)

# Combine all data
df = pd.DataFrame(games_played).join(deposits, how='outer').join(withdrawals, how='outer').fillna(0)

# Calculate Loyalty Points
df['Loyalty Points'] = (
    0.01 * df['total_deposit_amount'] +
    0.2 * df['Games Played'] -
    0.005 * df['total_withdrawal_amount'] +
    0.001 * (df['num_deposits'] - df['num_withdrawals']).clip(lower=0)
)

# Sort and save the result
df_sorted = df.sort_values(by='Loyalty Points', ascending=False)
df_sorted.to_excel("User_Loyalty_Points_Report.xlsx", sheet_name="Loyalty Points Report")


In [3]:
from google.colab import files
files.download("User_Loyalty_Points_Report.xlsx")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# New Section